In [1]:
import numpy as np
import pandas as pd
import sklearn


In [2]:
# On average, 5% of all transactions happen while the card holder is travelling.
f0 = pd.DataFrame({'Trav':['T',  'F'],
                   'Prob':[0.05, 0.95]})
print(f0,'\n------------------------------')

# More precisely, 1% of transactions are fraudulent when the card holder is travelling,where as only 0.4% of the transactions are fraudulent when she is not travelling.
f1 = pd.DataFrame({'Fraud':['T',  'F',  'T',   'F'],
                   'Trav': ['T',  'T',  'F',   'F'],
                   'Prob': [0.01, 0.99, 0.004, 0.996]
                  })
print(f1,'\n------------------------------')



# # On average, 5% of all transactions happen while the card holder is travelling.
# f2 = pd.DataFrame({'Trav':['T','F'],'Prob':[0.05,0.95]})
# print(f2,'\n------------------------------')

# not travelling, 10% of the fraudulent transactions are foreign purchases where as only 1% of the legitimate transactions are foreign purchases.
#  is travelling, 90% of the transactions are foreign purchases regardless of the legitimacy of the transactions.
f2 = pd.DataFrame({'Fraud':  ['T', 'F', 'T', 'F', 'T',  'F', 'T', 'F'], 
                    'FP':    ['T', 'T', 'F', 'F', 'T',  'T', 'F', 'F'],
                   'Trav':   ['T', 'T', 'T', 'T', 'F',  'F', 'F', 'F'], 
                    'Prob':  [0.9, 0.9, 0.1, 0.1, 0.1, 0.01, 0.9, 0.99]})
print(f2,'\n------------------------------')
# 60% of the population owns a computer and for those card holders
f3 = pd.DataFrame({'OC':['T','F'],'Prob':[0.6,0.4]})
print(f3,'\n------------------------------')
# Own Computer: 1% of their legitimate transactions are done over the internet, however this percentage increases to 2% for fraudulent transactions.
# not own computer:a mere 0.1% of their legitimate transactions is done over the internet, but that number increases to 1.1% for fraudulent transactions.
f4 = pd.DataFrame({'Fraud':  ['T', 'F', 'T',  'F',  'T',  'F', 'T', 'F'], 
                    'IP':    ['T', 'T', 'F',  'F',  'T',  'T', 'F', 'F'],
                    'OC':    ['T', 'T', 'T',  'T',  'F',  'F', 'F', 'F'],
                  'Prob':    [0.02,0.01,0.98,0.99,0.011,0.001,0.989,0.999]})
print(f4,'\n------------------------------')
# 10% of those who own a computer purchase with their credit card at least
# one computer related item as opposed to just 0.1% of those who don’t own any computer.
f5 = pd.DataFrame({'CRP': ['T','F','T','F'],
                   'OC':  ['T','T','F','F'],
                   'Prob':[0.1,0.9,0.001,0.999],
})
print(f5,'\n------------------------------')

  Trav  Prob
0    T  0.05
1    F  0.95 
------------------------------
  Fraud Trav   Prob
0     T    T  0.010
1     F    T  0.990
2     T    F  0.004
3     F    F  0.996 
------------------------------
  Fraud FP Trav  Prob
0     T  T    T  0.90
1     F  T    T  0.90
2     T  F    T  0.10
3     F  F    T  0.10
4     T  T    F  0.10
5     F  T    F  0.01
6     T  F    F  0.90
7     F  F    F  0.99 
------------------------------
  OC  Prob
0  T   0.6
1  F   0.4 
------------------------------
  Fraud IP OC   Prob
0     T  T  T  0.020
1     F  T  T  0.010
2     T  F  T  0.980
3     F  F  T  0.990
4     T  T  F  0.011
5     F  T  F  0.001
6     T  F  F  0.989
7     F  F  F  0.999 
------------------------------
  CRP OC   Prob
0   T  T  0.100
1   F  T  0.900
2   T  F  0.001
3   F  F  0.999 
------------------------------


In [3]:
def restrict(factor, variable, value):
    factor = factor.loc[factor[variable] == value]
#     del factor[variable]
    return factor
print(restrict(f4,'OC','T'))

  Fraud IP OC  Prob
0     T  T  T  0.02
1     F  T  T  0.01
2     T  F  T  0.98
3     F  F  T  0.99


In [4]:
def multiply(factor1,factor2):
    new_columns = list(set(f1.columns)|set(factor2.columns))
    if len(set(factor1.columns)) < len(set(factor2.columns)):
        factor1,factor2 = factor2,factor1
    common_keys = list(set(list(factor1.columns))  &  set(list(factor2.columns)))
    common_keys.remove('Prob')
    unique_keys = list(set(list(factor1.columns))  -  set(list(factor2.columns)))
    unique_keys.append('Prob')
    t1 = pd.merge(factor1,factor2,on=common_keys)
    t1['Prob']=t1['Prob_x']*t1['Prob_y']
    all_keys = t1.columns
#     for i in all_keys:
#         if i == None:
#             continue
#         if not i in unique_keys:
#             del t1[i]
    del t1['Prob_x']
    del t1['Prob_y']
    return t1
b=multiply(f0,f1)
print(b)

  Fraud Trav    Prob
0     T    T  0.0005
1     F    T  0.0495
2     T    F  0.0038
3     F    F  0.9462


In [5]:
def sumout(factor, variable):
    return factor.groupby(variable).sum()
d= sumout(b,'Fraud')
print(d)

         Prob
Fraud        
F      0.9957
T      0.0043


In [6]:
# def sumout(factor, variable):
#     factor=factor.drop(columns=[variable])
#     print(factor)
#     c = list(factor.columns).pop()
#     return factor.groupby(c)['Prob'].sum()

# d= sumout(b,'Fraud')
# print(d)

In [7]:
def normalize(factor):
    s = factor['Prob'].sum()
    
    factor['Prob']/=s
    return factor
print(normalize(b))
sum(list(normalize(b)['Prob']))

  Fraud Trav    Prob
0     T    T  0.0005
1     F    T  0.0495
2     T    F  0.0038
3     F    F  0.9462


1.0

In [8]:
def inference(factList, queryVar, HiddenVars, evidList):
    result = None
    # get a restricted factor set  
    restricted = []
    for factor in factList:
        for var in evidList.keys():
            if var not in factor.columns:
                continue
            factor = restrict(factor,var,evidList[var])
            restricted.append(factor)
    factList = [f for f in factList if not set(f.columns) in [set(rf.columns) for rf in restricted]]
    factList = factList+restricted

    for hv in HiddenVars:
        if hv in evidList:
            continue
        if hv in queryVar:
            print(hv,'in QueryVar')
            continue
        checkList = []
        newFactList = []
        for f in factList:
            if hv in f.columns:
                checkList.append(f)
            else:
                newFactList.append(f)
        if len(checkList) == 0:
            continue
        p = checkList[0]
        for i in range(1,len(checkList)):
            p = multiply(p,checkList[i])
        newFactList.append(p)
        factList = newFactList
        print('\n ----------------elimited var:%s --------\n'%hv)
        for i in factList:
            print(i)
    print('-----------------------')
    
    p = factList[0]
    for i in range(1,len(factList)):
        p = multiply(p,factList[i])
    ans = sumout(p,queryVar)
    ans = normalize(ans)
    
    return ans
    
#     for i in range(1,len(factList)):
#         p = multiply(p,factList[i])
#     return p

## B (1)

In [9]:
# B_1
a = multiply(f0,f1)
sumout(a,'Fraud')

,Prob
Fraud,
F,0.9957
T,0.0043


## B (2)

In [10]:
# B_2
fl=[f0,f1,f2,f3,f4,f5]
qv = ['Fraud']
oLHV = ['Trav', 'FP', 'Fraud', 'IP', 'OC', 'CRP']
eL = {'IP':'F', 'CRP': 'T', 'FP': 'T'}
ans= inference(fl,qv,oLHV,eL)
print(ans)



 ----------------elimited var:Trav --------

  OC  Prob
0  T   0.6
1  F   0.4
  Fraud IP OC   Prob
2     T  F  T  0.980
3     F  F  T  0.990
6     T  F  F  0.989
7     F  F  F  0.999
  CRP OC   Prob
0   T  T  0.100
2   T  F  0.001
  Fraud FP Trav      Prob
0     T  T    T  0.000450
1     F  T    T  0.044550
2     T  T    F  0.000380
3     F  T    F  0.009462
Fraud in QueryVar

 ----------------elimited var:OC --------

  Fraud FP Trav      Prob
0     T  T    T  0.000450
1     F  T    T  0.044550
2     T  T    F  0.000380
3     F  T    F  0.009462
  Fraud IP OC CRP      Prob
0     T  F  T   T  0.058800
1     F  F  T   T  0.059400
2     T  F  F   T  0.000396
3     F  F  F   T  0.000400
-----------------------
           Prob
Fraud          
F      0.985016
T      0.014984


## C

In [11]:
# C
fl=[f0,f1,f2,f3,f4,f5]
qv = ['Fraud']
oLHV = ['Trav', 'FP', 'Fraud', 'IP', 'OC', 'CRP']

eL = {'IP':'F', 'CRP': 'T', 'FP': 'T','Trav':'T'}

ans=inference(fl, qv,oLHV,eL)
print(ans)

Fraud in QueryVar

 ----------------elimited var:OC --------

  Trav  Prob
0    T  0.05
  Fraud Trav  Prob
0     T    T  0.01
1     F    T  0.99
  Fraud FP Trav  Prob
0     T  T    T  0.90
1     F  T    T  0.90
4     T  T    F  0.10
5     F  T    F  0.01
  Fraud FP Trav  Prob
0     T  T    T   0.9
1     F  T    T   0.9
  Fraud IP OC CRP      Prob
0     T  F  T   T  0.058800
1     F  F  T   T  0.059400
2     T  F  F   T  0.000396
3     F  F  F   T  0.000400
-----------------------
         Prob
Fraud        
F      0.9901
T      0.0099


## D


Use it to buy something with  a computer related purchase.
Then make the internet purchase.
Then 

In [12]:
# df2 = pd.DataFrame({'Trav':  ['F', 'F', 'T', 'T', 'F', 'F', 'T', 'T'], 
#                     'Fraud': ['T', 'F', 'T', 'F', 'T', 'F', 'T', 'F'], 
#                     'FP':    ['T','T','T','T', 'F', 'F', 'F', 'F'], 
#                     "F":     [0.1,0.01,0.9,0.9, 0.9,0.99,0.1,0.1]})

# pd.merge(f3,df2,on=['FP','Fraud','Trav'])

In [13]:
# df4 = pd.DataFrame({'OC': ['T', 'T', 'F', 'F', 'T', 'T', 'F', 'F'], 
#                     'Fraud': ['F', 'T', 'F', 'T', 'F', 'T', 'F', 'T'],
#                     'IP': ['F', 'F', 'F', 'F', 'T', 'T', 'T', 'T'], 
#                     "F": [0.99, 0.98, 0.999, 0.989, 0.01, 0.02, 0.001, 0.011]})

# pd.merge(f5,df4,on=['IP','Fraud','OC'])

In [14]:
# df5 = pd.DataFrame({'OC': ['T', 'F', 'T', 'F'],'CRP': ['T', 'T', 'F', 'F'], "F": [0.1, 0.001, 0.9, 0.999]})
# pd.merge(f6,df5,on=['CRP','OC'])